In [ ]:
using CSV
using DataFrames
df= DataFrame(CSV.File("/home/jkambulo/projects/def-pnroy/jkambulo/dmrg/output_data/ED_benchmark_v3_graham.csv"))


In [ ]:
data = Matrix(df[(df.Nsites .== 6) .& (df.mmax .== 5),:][!,[string("E",i) for i in 1:20]])

while true
    losses = []
    loss_arguments = []
    for g_index in 2:size(data)[1]-2
        for energy_index2 in 1:size(data)[2]
            for energy_index in 1:size(data)[2]
                tmp_data = copy(data[:, [energy_index, energy_index2]])
                diff2_data = abs.(diff(diff(tmp_data, dims=1), dims=1))
                loss = sum(diff2_data)

                tmp_data[g_index:end,:] = reverse(tmp_data[g_index:end,:], dims=2)
                new_loss = sum(abs.(diff(diff(tmp_data,dims=1),dims=1)))

                # print(new_loss-loss)
                if round(new_loss - loss, digits=5) < 0
                    push!(loss_arguments,[g_index, energy_index, energy_index2])
                    push!(losses, new_loss-loss)
                end
            end
        end
    end  

    

    if length(losses) == 0
        break
    end
    i = argmin(losses)
    println(length(losses)," ", losses[i]," ",loss_arguments[i])
    data[loss_arguments[i][1]:end, loss_arguments[i][2:3]] = data[loss_arguments[i][1]:end, reverse(loss_arguments[i][2:3])]
end